# Task 1: SPX

In [75]:
import pandas as pd
import numpy as np
import datetime

In [76]:
#read data
data = pd.read_parquet('data/spx_ivs_2023-02.parquet')

# convert date
data['date'] = pd.to_datetime(data['loctimestamp'], unit='ms').dt.date

data.head()

,Symbol,loctimestamp,putcall,daystomaturity,implVol,implPrice,strike,forwardMoneyness,normalizedMoneyness,underlyingprice,underlyingforwardprice,date
0,^SPX,2023-02-01,P,30,0.546362,0.162479,2572.16,0.620158,-10.00,4138.45,4147.58,2023-02-01
1,^SPX,2023-02-01,P,30,0.537310,0.176469,2603.06,0.627610,-9.75,4138.45,4147.58,2023-02-01
2,^SPX,2023-02-01,P,30,0.528999,0.195240,2634.34,0.635152,-9.50,4138.45,4147.58,2023-02-01
3,^SPX,2023-02-01,P,30,0.520193,0.214198,2666.00,0.642784,-9.25,4138.45,4147.58,2023-02-01
4,^SPX,2023-02-01,P,30,0.510792,0.232522,2698.03,0.650507,-9.00,4138.45,4147.58,2023-02-01


In [77]:
# get relevant data
results = []

for day in pd.date_range(start='2023-02-01', end='2023-02-28'):
    date = day.date()

    data_day = data[data['date'] == date]

    if data_day.empty:
        continue

    F = data_day['underlyingforwardprice'].iloc[0]
    S_t = data_day['underlyingprice'].iloc[0]

    call_strikes = data_day[data_day['putcall'] == 'C']['strike'].unique()
    put_strikes  = data_day[data_day['putcall'] == 'P']['strike'].unique()

    call_strike = call_strikes[np.abs(call_strikes - F).argmin()]
    put_strike  = put_strikes[np.abs(put_strikes - F).argmin()]

    call_price = data_day[(data_day['putcall'] == 'C') & (data_day['strike'] == call_strike)]['implPrice'].mean()
    put_price  = data_day[(data_day['putcall'] == 'P') & (data_day['strike'] == put_strike)]['implPrice'].mean()

    K = (call_strike + put_strike) / 2

    results.append({
        'date': date,
        'forward_price': F,
        'underlying_price': S_t,
        'strike': K,
        'call_strike': call_strike,
        'call_price': call_price,
        'put_strike': put_strike,
        'put_price': put_price
    })

print(len(results))

19


In [80]:
#
p = {'date': datetime.date(2023, 2, 1), 'forward_price': 4147.58, 'underlying_price': 4138.45, 'strike': 4122.955, 'call_strike': 4147.58, 'call_price': 78.7447, 'put_strike': 4098.33, 'put_price': 59.3535}



def bkm_variance(data):
    S_t = data['underlying_price']
    K = data['strike']
    C = data['call_price']
    P = data['put_price']
    print(K)

    integrand_c = 2 * (1 - np.log(K / S_t)) *  C / (S_t ** 2)

    var_call = np.trapz(integrand_c, K)

    integrand_p = 2 * (1 - np.log(S_t / K)) *  P / (S_t ** 2)
    var_put = np.trapz(integrand_p, K)

    variance = var_call + var_put

    return variance



print(bkm_variance(p))


4122.955


ValueError: diff requires input that is at least one dimensional